
# Análisis de Secuencias Nucleotídicas de Integronas de Shewanella

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [15]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display
import pkg_resources

def instalar_librerias():
    librerias = [
        "import_ipynb",
        "nbconvert",
        "ipywidgets",
        "biopython",
        "pandas",
        "numpy",
        "matplotlib",
        "tqdm",
        "odfpy",
        "requests",
        "gdown",
        "mafft",
        "clustalo",
        "muscle",
        "fasttree"
    ]

    barra_progreso = widgets.IntProgress(
        value=0,
        min=0,
        max=len(librerias),
        bar_style='info',
        orientation='horizontal'
    )

    etiqueta = widgets.Label(value='Iniciando instalación...')
    display(widgets.VBox([etiqueta, barra_progreso]))

    # Instalación de fasttree y mafft con apt-get (solo ejecutar en Colab)
    subprocess.run(["apt-get", "install", "fasttree", "mafft"])

    for libreria in librerias:
        try:
            pkg_resources.require(libreria)
            etiqueta.value = f"{libreria} ya está instalado."
        except pkg_resources.DistributionNotFound:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
        barra_progreso.value += 1

    etiqueta.value = 'Instalación completada.'

# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)

# Enlazar el botón con la función de instalación
boton_instalar.on_click(lambda b: instalar_librerias())

# Mostrar botón
display(boton_instalar)

#instalar_librerias()

Output()

Button(description='Instalar Librerías', style=ButtonStyle())

In [16]:
#@title combina varias archivos fasta

def combinar_en_multifasta(rutas_archivos_fasta, archivo_multifasta):
    with open(archivo_multifasta, 'w') as archivo_salida:
        for ruta_fasta in rutas_archivos_fasta:
            with open(ruta_fasta, 'r') as archivo_entrada:
                archivo_salida.write(archivo_entrada.read() + "\n")

# Combina los archivos FASTA en un solo archivo multifasta
#archivo_multifasta = "secuencias_combinadas.fasta"
#combinar_en_multifasta(rutas_archivos_fasta, archivo_multifasta)


In [17]:
#@title alineacion con mafft

from Bio import SeqIO
from Bio.Align.Applications import MafftCommandline
import multiprocessing
from ipywidgets import IntProgress, VBox, Label
from IPython.display import display
import os
import time
import threading
from ipywidgets import Text

def dividir_archivo_multifasta(archivo_multifasta, num_secuencias_por_archivo):
    with open(archivo_multifasta, "r") as handle:
        secuencias = list(SeqIO.parse(handle, "fasta"))

    archivos_divididos = []
    for i in range(0, len(secuencias), num_secuencias_por_archivo):
        archivo_parte = f"parte_{i//num_secuencias_por_archivo}.fasta"
        with open(archivo_parte, "w") as output_handle:
            SeqIO.write(secuencias[i:i + num_secuencias_por_archivo], output_handle, "fasta")
        archivos_divididos.append(archivo_parte)

    return archivos_divididos

def alinear_con_mafft(archivo_fasta):
    archivo_salida = archivo_fasta.replace(".fasta", "_alineado.fasta")
    mafft_cline = MafftCommandline(input=archivo_fasta, auto=True)
    stdout, stderr = mafft_cline()
    with open(archivo_salida, "w") as output_handle:
        output_handle.write(stdout)
    return archivo_salida

def alinear_archivos_en_paralelo(archivos_divididos):
    pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
    resultados = pool.map(alinear_con_mafft, archivos_divididos)
    pool.close()
    pool.join()
    return resultados

def mostrar_indicador_actividad(label, intervalo=5):
    global proceso_activo
    proceso_activo = True
    contador = 0
    while proceso_activo:
        time.sleep(intervalo)
        contador += intervalo
        if proceso_activo:
            label.value = f"El proceso sigue activo... {contador} segundos"

def alinear_secuencias(archivo_multifasta, num_secuencias_por_archivo):
    archivos_divididos = dividir_archivo_multifasta(archivo_multifasta, num_secuencias_por_archivo)

    # Inicializar barra de progreso e indicador de actividad
    progress_bar = IntProgress(min=0, max=len(archivos_divididos))
    progress_label = Label('Iniciando alineamiento...')
    activity_label = Text(value='El proceso ha comenzado.', disabled=True)
    display(VBox([progress_label, progress_bar, activity_label]))

    # Iniciar indicador de actividad
    hilo_indicador = threading.Thread(target=mostrar_indicador_actividad, args=(activity_label,))
    hilo_indicador.start()

    # Alineamiento en paralelo
    for i, _ in enumerate(alinear_archivos_en_paralelo(archivos_divididos), 1):
        progress_bar.value = i
        progress_label.value = f"Alineando archivo {i} de {len(archivos_divididos)}"

    # Finalizar indicador de actividad
    global proceso_activo
    proceso_activo = False
    hilo_indicador.join()

    progress_label.value = 'Alineamiento completado.'
    activity_label.value = 'Proceso finalizado.'

# Uso de la función
archivo_multifasta = "tu_archivo_combinado.fasta"  # Reemplaza con la ruta de tu archivo
num_secuencias_por_archivo = 100  # Ajusta según tus necesidades
#alinear_secuencias(archivo_multifasta, num_secuencias_por_archivo)



In [18]:
#@title contruir el arbol filogenetico
from Bio.Align.Applications import MafftCommandline
from Bio import AlignIO
from Bio import Phylo
from Bio.Phylo.Applications import FastTreeCommandline
import os
import subprocess

def construir_arbol_con_mafft(ruta_alineamiento):
    if not os.path.exists(ruta_alineamiento):
        print(f"El archivo {ruta_alineamiento} no existe.")
        return None

    archivo_arbol = "arbol_mafft.nwk"

    try:
        # Ejecutar FastTree
        comando_fasttree = ["fasttree", "-out", archivo_arbol, ruta_alineamiento]
        subprocess.run(comando_fasttree, check=True)

        # Leer el árbol
        arbol = Phylo.read(archivo_arbol, "newick")
        return arbol
    except Exception as e:
        print(f"Ocurrió un error al construir el árbol: {e}")
        return None

# Uso de la función
#ruta_alineamiento = "/content/parte_0_alineado.fasta"  # Reemplaza con la ruta de tu archivo alineado
#construir_arbol_con_mafft(ruta_alineamiento)


In [19]:
#@title graficar regiones conservadas

from Bio import AlignIO
import matplotlib.pyplot as plt
import numpy as np

def graficar_conservacion(archivo_alineamiento):
    alineamiento = AlignIO.read(archivo_alineamiento, "fasta")
    num_secuencias = len(alineamiento)
    longitud_secuencia = alineamiento.get_alignment_length()

    matriz_conservacion = np.zeros(longitud_secuencia)

    for i in range(longitud_secuencia):
        columna = alineamiento[:, i]
        residuo_mas_comun = max(set(columna), key=columna.count)
        conservacion = columna.count(residuo_mas_comun) / num_secuencias
        matriz_conservacion[i] = conservacion

    plt.figure(figsize=(12, 4))
    plt.plot(matriz_conservacion, color='blue', lw=2)
    plt.xlabel('Posición en la Secuencia')
    plt.ylabel('Conservación')
    plt.title('Perfil de Conservación del Alineamiento')
    plt.ylim(0, 1)
    plt.show()

# Ejemplo de uso
#graficar_conservacion('parte_0_alineado.fasta')


In [20]:
from Bio import AlignIO
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# Función para calcular la entropía de cada posición
def calcular_entropia(archivo_alineamiento):
    alineamiento = AlignIO.read(archivo_alineamiento, "fasta")
    longitud = alineamiento.get_alignment_length()
    entropias = np.zeros(longitud)

    for i in range(longitud):
        columna = [record.seq[i] for record in alineamiento]
        entropias[i] = stats.entropy(np.unique(columna, return_counts=True)[1], base=2)

    return entropias

# Función para graficar la entropía
def graficar_entropia(archivo_alineamiento):
    entropias = calcular_entropia(archivo_alineamiento)
    plt.figure(figsize=(12, 4))
    plt.plot(entropias, color='red', lw=2)
    plt.xlabel('Posición en la Secuencia')
    plt.ylabel('Entropía')
    plt.title('Perfil de Entropía del Alineamiento')
    plt.ylim(0, max(entropias))
    plt.show()

# Función para generar un mapa de calor de conservación
def mapa_calor_conservacion(archivo_alineamiento):
    alineamiento = AlignIO.read(archivo_alineamiento, "fasta")
    num_secuencias = len(alineamiento)
    longitud = alineamiento.get_alignment_length()
    matriz_conservacion = np.zeros((num_secuencias, longitud))

    for i, record in enumerate(alineamiento):
        for j in range(longitud):
            matriz_conservacion[i, j] = alineamiento[:, j].count(record.seq[j]) / num_secuencias

    plt.figure(figsize=(12, 6))
    plt.imshow(matriz_conservacion, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Grado de Conservación')
    plt.xlabel('Posición en la Secuencia')
    plt.ylabel('Secuencia')
    plt.title('Mapa de Calor de Conservación de Secuencias')
    plt.show()

from Bio import AlignIO
import numpy as np
import matplotlib.pyplot as plt

def graficar_barras_conservacion(archivo_alineamiento):
    alineamiento = AlignIO.read(archivo_alineamiento, "fasta")
    num_secuencias = len(alineamiento)
    longitud_secuencia = alineamiento.get_alignment_length()

    porcentajes_conservacion = np.zeros(longitud_secuencia)

    for i in range(longitud_secuencia):
        columna = alineamiento[:, i]
        residuo_mas_comun = max(set(columna), key=columna.count)
        conservacion = (columna.count(residuo_mas_comun) / num_secuencias) * 100
        porcentajes_conservacion[i] = conservacion

    plt.figure(figsize=(15, 4))
    plt.bar(range(longitud_secuencia), porcentajes_conservacion, color='green')
    plt.xlabel('Posición en la Secuencia')
    plt.ylabel('Porcentaje de Conservación (%)')
    plt.title('Conservación por Posición en el Alineamiento')
    plt.ylim(0, 100)
    plt.show()

#graficar_entropia('tu_archivo_alineado.fasta')
#mapa_calor_conservacion('tu_archivo_alineado.fasta')
#graficar_barras_conservacion('tu_archivo_alineado.fasta')
